# Лаба 2 - Линейная регрессия

## Задача
В этом наборе данных вам предстоит решить актуальную задачу для поисковиков: **нужно оценить насколько привлекателен веб-ресурс по некоторому набору факторов**.

В наборе данных представлено 8000 записей о различных анонимизированных доменах и соответствующие оценки привлекательности(числовые)

Нужно понять насколько домен привлекателен по остальным факторам.

## Описание столбцов

| столбец                | описание                                            |
|------------------------|-----------------------------------------------------|
| category               | категория к которой относится сайт                  |
| clicks                 | кол-во кликов по домену                             |
| likes                  | кол-во лайков поставленных домену                   |
| buys                   | кол-во покупок совершенных на домене                |
| 4xx_errors             | кол-во ошибок с кодом 4хх за последние 6 мес        |
| 5xx_errors             | кол-во ошибок с кодом 5хх за последние 6 мес        |
| complaints_count       | кол-во жалоб на домен                               |
| average_dwelltime      | среднее время проведенное пользователем на домене   |
| date_of_registration   | дата регистрации домена                             |
| source_attractiveness  | привлекательность домена (таргет)                   |

### Пояснение к задаче
Ваша задача предстоит не только в написании кода обучения модели. Глобально вы должны предоставить ноутбук с полноценным анализом данных, очисткой данных, сделать выводы на основе графического анализа.

За отсутствие надлежащего оформления будут снижаться баллы. В критических случаях - лаба не будет принята.

In [94]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [95]:
data_path = 'train.csv'
df = pd.read_csv(data_path)

# set up id
df.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
df.set_index('id', inplace=True)

Посмотрим на некоторые распределения в датасете:

In [101]:
categories = df.groupby('category')
categories_count = categories.count().max(axis=1)
categories_likes = categories['likes'].sum()
categories_buys = categories['buys'].sum()
categories_clicks = categories['clicks'].sum()

categories_bars = make_subplots(rows=4)

categories_bars.add_trace(
    go.Bar(
        name='Кол-во доменов',
        x=categories_count.index,
        y=categories_count.values,
    ),
    row=1, col=1,
)
categories_bars.add_trace(
    go.Bar(
        name='Кол-во кликов',
        x=categories_clicks.index,
        y=categories_clicks.values,
    ),
    row=2, col=1,
)
categories_bars.add_trace(
    go.Bar(
        name='Кол-во лайков',
        x=categories_likes.index,
        y=categories_likes.values,
    ),
    row=3, col=1,
)
categories_bars.add_trace(
    go.Bar(
        name='Кол-во покупок',
        x=categories_buys.index,
        y=categories_buys.values,
    ),
    row=4, col=1,
)

categories_bars.update_layout(
    title='Разбиение доменов на категории',
    barmode='group',
    height=800,
)

categories_bars.show()



# fig = make_subplots(rows=1, cols=2)

# fig.add_trace(
#     go.Scatter(x=[1, 2, 3], y=[4, 5, 6]),
#     row=1, col=1
# )

# fig.add_trace(
#     go.Scatter(x=[20, 30, 40], y=[50, 60, 70]),
#     row=1, col=2
# )

# fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
# fig.show()


# h = go.Figure(
#     go.Histogram(
#         x=,
#     )
# )

# h.show()

# df.describe()

## Final model submit function

Принимает на вход путь к набору данных в формате `.csv`

На выходе получается файл с предиктами в формате `.csv`

Пример содержания:
```
0.432
0.412
0.312
```

Реализация модели - между ними

In [30]:
# def predict(data_path):
#     data = pd.read_csv(data_path)
#     # Add your implementation
#     predicts = #...
#     with open('output.csv', 'w') as file:
#         for predict in predicts:
#             file.write(str(predict) + '\n')

SyntaxError: invalid syntax (3716318233.py, line 4)